In [ ]:
import os
import time
import requests

ELEFAN_PRODUCT_API_URL = os.environ.get("API_URL")
OFF_API_URL = "https://fr.openfoodfacts.org/api/v2/product"
OFF_API_UPDATE_URL = "https://world.openfoodfacts.org/cgi/product_jqm2.pl"
OFF_KEY = "OFF"

ELEFAN_FAMILLE_CODE_TO_EXCLUDE = [
    21,  # bebe (inclus nourriture...)
    22,  # hygiène, beauté
    23,  # entretien, nettoyage
    24,  # cuisine
    25,  # santé
    31,  # journaux
    41,  # maison
    42,  # papeterie
    43,  # textile
    44,  # animalerie
    45,  # jardinerie
    90,  # jeux
]

In [ ]:
"""
Récupérer les produits depuis Elefan (via l'API)
"""
print("Elefan Grenoble")
elefan_product_list = list()

response = requests.get(ELEFAN_PRODUCT_API_URL)
elefan_product_list = response.json()
print("Nombre d'articles (dans KASO) :", len(elefan_product_list))
# print(elefan_product_list[0])

elefan_product_actifs_list = [p for p in elefan_product_list if p['status'] == 'ACTIF']
print("Nombre d'articles actifs (status = ACTIF dans KASO) :", len(elefan_product_actifs_list))

elefan_product_actifs_ean_list = [p for p in elefan_product_actifs_list if (len(str(p['code'])) in [8, 13])]
print("Nombre d'articles actifs avec un code-barres EAN (8 ou 13 chiffres) :", len(elefan_product_actifs_ean_list))

elefan_product_actifs_ean_filtered_list = [p for p in elefan_product_actifs_ean_list if p['famille']['code'] not in ELEFAN_FAMILLE_CODE_TO_EXCLUDE]
print("Nombre d'articles actifs avec un code-barres EAN et filtrés (catégories) :", len(elefan_product_actifs_ean_filtered_list))

In [ ]:
print(f"Lien avec {OFF_KEY} (pour les {len(elefan_product_actifs_ean_filtered_list)} articles filtrés)")
found = 0

for index, elefan_product in enumerate(elefan_product_actifs_ean_filtered_list):
    response = requests.get(f"{OFF_API_URL}/{elefan_product['code']}.json")
    if response.status_code == 200:
        found += 1
        elefan_product_actifs_ean_filtered_list[index][OFF_KEY] = response.json()
    if (index % 10) == 0:
        time.sleep(1)
    if (index % 100) == 0:
        print(f"progress: {index}...")

print("Trouvés", found, "/", len(elefan_product_actifs_ean_filtered_list))

In [ ]:
from collections import Counter

elefan_product_actifs_ean_filtered_found_list = [p for p in elefan_product_actifs_ean_filtered_list if OFF_KEY in p]
print(f"Sur les {len(elefan_product_actifs_ean_filtered_found_list)} produits trouvés dans {OFF_KEY}...")

for key in ["product_name", "quantity", "image_url", "categories", "brands", "labels", "origins", "packaging", "stores"]:
    print(f"Avec {key} :", len([p for p in elefan_product_actifs_ean_filtered_found_list if (key in p[OFF_KEY]['product'] and p[OFF_KEY]['product'][key])]))

for key in ["nutriscore_grade", "ecoscore_grade"]:
    key_list = list()
    for p in elefan_product_actifs_ean_filtered_found_list:
        if key in p['OFF']['product']:
            key_list.append(p[OFF_KEY]['product'][key])
    print(f"{key} :", Counter(key_list))

for key in ["nova_groups_tags"]:  # always length 1
    key_list = list()
    for p in elefan_product_actifs_ean_filtered_found_list:
        if key in p['OFF']['product']:
            key_list.extend(p[OFF_KEY]['product'][key])
    print(f"{key} :", Counter(key_list))

In [ ]:
# Send store information
print(len(elefan_product_actifs_ean_filtered_found_list))

for index, product in enumerate(elefan_product_actifs_ean_filtered_found_list):
    STORE_ELEFAN = "L'Éléfàn"
    COUNTRY_FRANCE = "France"
    OFF_USERNAME = os.environ.get("OFF_USERNAME")
    OFF_PASSWORD = os.environ.get("OFF_PASSWORD")
    response = requests.get(f"{OFF_API_UPDATE_URL}?code={product['code']}&add_stores={STORE_ELEFAN}&add_countries={COUNTRY_FRANCE}&user_id={OFF_USERNAME}&password={OFF_PASSWORD}")
    print(product['code'], response.json())
    if (index % 10) == 0:
        time.sleep(1)
    if (index % 100) == 0:
        print(f"progress: {index}...")